In [2]:
import pandas as pd

df1 = pd.read_csv("data/train_dataset.csv")
df2 = pd.read_csv("data/valid_dataset.csv")
df3 = pd.read_csv("data/test_dataset.csv")

df = pd.concat([df1, df2, df3]).reset_index(drop=True)
df = df.loc[:, "Target Sequence"].drop_duplicates().reset_index(drop=True)
df

0       MLKFKYGARNPLDAGAAEPIASRASRLNLFFQGKPPFMTQQQMSPL...
1       MRHSKRTYCPDWDDKDWDYGKWRSSSSHKRRKRSHSSAQENKRCKY...
2       MRHSKRTHCPDWDSRESWGHESYRGSHKRKRRSHSSTQENRHCKPH...
3       MAPFLRIAFNSYELGSLQAEDEANQPFCAVKMKEALSTERGKTLVQ...
4       MVVFNGLLKIKICEAVSLKPTAWSLRHAVGPRPQTFLLDPYIALNV...
                              ...                        
3062    MGEGGAAAALVAAAAAAAAAAAAVVAGQRRRRLGRRARCHGPGRAA...
3063    MAAAAAAGPEMVRGQVFDVGPRYTNLSYIGEGAYGMVCSAYDNLNK...
3064    MDRLDANVSSNEGFGSVEKVVLLTFFAMVILMAILGNLLVMVAVCR...
3065    MATALMAVVLRAAAVAPRLRGRGGTGGARRLSCGARRRAARGTSPG...
3066    MAWALLLLTLLTQGTGSWAQSALTQPPSASGSPGQSVTISCTGTSS...
Name: Target Sequence, Length: 3067, dtype: object

In [3]:
import pickle
import pandas as pd
import transformers
from tqdm import tqdm
from transformers import AutoModel, BertTokenizer, RobertaTokenizer

prot_tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert_bfd", do_lower_case=False)
prot_encoder = AutoModel.from_pretrained("Rostlab/prot_bert_bfd").to("cuda")
prot_encoder.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30, 1024, padding_idx=0)
    (position_embeddings): Embedding(40000, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-29): 30 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.0, inpla

In [4]:
# for max_length in [256, 512, 768, 1024, 2048]:
for max_length in [545]:
    results = {}

    for data in tqdm(df, total=len(df)):
        seq = prot_tokenizer(" ".join(data), max_length=max_length + 2, truncation=True, return_tensors="pt").to("cuda")
        a = prot_encoder(**seq)
        a = a.last_hidden_state.detach().to("cpu")
        results[data[:20]] = a[:, 0]

    with open(f"prot_feat/{max_length}_cls.pkl", "wb") as f:
        pickle.dump(results, f)

  0%|          | 0/3067 [00:00<?, ?it/s]

100%|██████████| 3067/3067 [01:21<00:00, 37.77it/s]
